In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# sklearn & spaCy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
import spacy

# visualization
import matplotlib.pyplot as plt
%matplotlib inline

# dataprep
from dataprep import clean

In [7]:
# load file
file = 'chatgpt-reddit-comments.csv'
df = pd.read_csv(file, usecols = ['comment_body'])

# drop NULLs & duplicates (if any)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

# reset index
df.reset_index(drop=True,inplace=True)

# clean data
df = clean.clean_text(df, 'comment_body')

# lemmetize
lem = spacy.load('en_core_web_sm', disable = ['parser','ner'])

def lemmetize(txt):
    doc = lem(txt)
    return (" ".join([token.lemma_ for token in doc]))

df['comment_body'] = df['comment_body'].apply(lambda x: lemmetize(x))

# view
df.head()

C:\Users\Sico\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:6890: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
C:\Users\Sico\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:6890: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
C:\Users\Sico\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\core.py:6890: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a

,comment_body
0,shocked day need clickbait
1,angry right waste time read post sub clickbait...
2,chatgpt karma whore folk think stream think bu...
3,work ngl
4,certify must see moment really shock core


In [8]:
# feature extraction
vec = TfidfVectorizer(stop_words='english')
vec.fit(df.comment_body.values)
features = vec.transform(df.comment_body.values)

In [ ]:
# model training
cls = MiniBatchKMeans(n_clusters=3, random_state=0)
cls.fit(features)

# prediction
pred = cls.predict(features)

# reduction of features to 2D
pca = PCA(n_components=2, random_state=0)
reduced_features = pca.fit_transform(features.toarray())

# reduction of cluster-centers to 2D
red_cluster_centers = pca.transform(cls.cluster_centers_)

# plotting
plt.figure(figsize=(15,10), dpi=80)
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=pred)
plt.scatter(red_cluster_centers[:,0], red_cluster_centers[:,1], marker='X', s=150, c='b')
plt.show()

C:\Users\Sico\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
